<hr>
<center> Learning Element for Control Systems - Autumn Semester 2024 <br>
<b> Learning Element 1: Solution </b> <br>
Prof. Dr. Florian Dörfler <br>
Automatic Control Laboratory, ETH Zurich </center>
<hr>

In this notebook, you will find different code fragments to complete Learning Element 1. <b>

<!-- <blockquote> -->
<b>Activity:</b> Execute the code cells below so that the necessary libraries get imported.
<!-- </blockquote> -->

In [2]:
import numpy as np # the standard library for numerics, vectors, matrices
import control as ct # the standard library for basic operations for analysis and design of feedback control systems
import matplotlib.pyplot as plt # a comprehensive library for creating static, animated, and interactive visualizations
import sympy as sp # the standard library for symbolic mathematical computations

<br>

<b> Code fragment for T1.1-i) linearization </b>

In [63]:
# define symbolic variables
x, y, z, xdot, ydot, zdot, phi, theta, psi, p, q, r = sp.symbols('x, y, z, xdot, ydot, zdot, phi, theta, psi, p, q, r')
u1,u2,u3,u4, = sp.symbols('u1, u2, u3, u4')
m, g, Ix, Iy, Iz, kx, ky, kz, kp, kq, kr = sp.symbols('m, g, Ix, Iy, Iz, kx, ky, kz, kp, kq, kr')
xd, yd, zd, psid = sp.symbols('xd, yd, zd, psid')

# define symbolic state, input & equilibrium vector
states = sp.Array([[x], [y], [z], [xdot], [ydot], [zdot], [phi], [theta], [psi], [p], [q], [r]])
inputs = sp.Array([[u1], [u2], [u3], [u4]])

# nonlinear model
xddot = 1/m*((sp.cos(phi)*sp.sin(theta)*sp.cos(psi) + sp.sin(phi)*sp.sin(psi))*u1 - kx*xdot)
yddot = 1/m*((sp.cos(phi)*sp.sin(theta)*sp.sin(psi) - sp.sin(phi)*sp.cos(psi))*u1 - ky*ydot)
zddot = 1/m*(sp.cos(phi)*sp.cos(theta)*u1 - m*g - kz*ydot)
phidot = p + sp.sin(phi)*sp.tan(theta)*q + sp.cos(phi)*sp.tan(theta)*r
thetadot = sp.cos(phi)*q - sp.sin(phi)*r
psidot = sp.sin(phi)/sp.cos(theta)*q + sp.cos(phi)/sp.cos(theta)*r
pdot = 1/Ix*((Iy - Iz)*q*r + u2 - kp*p)
qdot = 1/Iy*((Iz - Ix)*p*r + u3 - kq*q)
rdot = 1/Iz*((Ix - Iy)*p*q + u4 - kr*r)

drone = sp.Matrix([[xdot],[ydot],[zdot],[xddot],[yddot],[zddot],[phidot],[thetadot],[psidot],[pdot],[qdot],[rdot]])

# sp.pprint(drone)             
                   
# compute symbolic state-space matrices
A_jacobain = drone.jacobian(states)
B_jacobain = drone.jacobian(inputs)

# sp.pprint(A_jacobain)        
# sp.pprint(A_jacobain)

# compute numeric state-space matrices with dtype float
eq_dict = {x:xd, y:yd, z:zd, 
           xdot:0, ydot:0, zdot:0, 
           phi:0, theta:0, psi:psid, 
           p:0, q:0, r:0, 
           u1:m*g, u2:0, u3:0, u4:0}

A_linearized = A_jacobain.subs(eq_dict)
B_linearized = B_jacobain.subs(eq_dict)

constants = {'xd':1, 'yd':1, 'zd':1,
             'psid':sp.pi/4, 'm':0.3, 'g':9.81, 
             'kx':4.5e-3, 'ky':4.5e-3, 'kz':4.5e-3, 
             'kp':4.5e-4, 'kq':4.5e-4, 'kr':4.5e-4, 
             'Ix':1.5e-5,'Iy':1.5e-5,'Iz':3e-5}

A = A_linearized.subs(constants)
B = B_linearized.subs(constants)

C = [[1,0,0,0,0,0,0,0,0,0,0,0],
     [0,1,0,0,0,0,0,0,0,0,0,0],
     [0,0,1,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,1,0,0,0,0,0]]
D = [[0,0,0,0],
    [0,0,0,0],
    [0,0,0,0],
    [0,0,0,0]]


# print system matrices A, B
np.set_printoptions(precision=4,suppress=True,linewidth=np.inf)
print('The system matrix A : ')
sp.pprint(A)
print('\nThe system matrix B :')
sp.pprint(B)

The system matrix A : 
⎡0  0  0    1       0     0      0         0      0    0      0      0  ⎤
⎢                                                                       ⎥
⎢0  0  0    0       1     0      0         0      0    0      0      0  ⎥
⎢                                                                       ⎥
⎢0  0  0    0       0     1      0         0      0    0      0      0  ⎥
⎢                                                                       ⎥
⎢0  0  0  -0.015    0     0  4.905⋅√2   4.905⋅√2  0    0      0      0  ⎥
⎢                                                                       ⎥
⎢0  0  0    0     -0.015  0  -4.905⋅√2  4.905⋅√2  0    0      0      0  ⎥
⎢                                                                       ⎥
⎢0  0  0    0     -0.015  0      0         0      0    0      0      0  ⎥
⎢                                                                       ⎥
⎢0  0  0    0       0     0      0         0      0    1      0      0  ⎥
⎢              

<br>

<b> Code fragment for T1.1-ii) calculating transfer functions </b>

In [70]:
# create state-space system & transfer function
sys2_ss = ct.ss(A,B,C,D) # create a state-space object
tf = ct.ss2tf(sys2_ss) # create transfer function

sys2_ss
tf

TransferFunction([[array([0.]), array([462447.8349,      0.    ]), array([     0.    , 462447.8349,      0.    ]), array([-0.,  0.])], [array([0.]), array([-462447.8349,       0.    ]), array([     0.    , 462447.8349,      0.    ]), array([0., 0.])], [array([  3.3333, 100.05  ,   1.5   ,  -0.    ]), array([6936.7175]), array([-6936.7175]), array([-0., -0.])], [array([0.]), array([66666.6667,     0.    ,     0.    ,     0.    ]), array([0.]), array([0.])]], [[array([1.]), array([ 1.   , 30.015,  0.45 ,  0.   ,  0.   ,  0.   ]), array([ 1.   , 30.015,  0.45 ,  0.   ,  0.   ,  0.   ]), array([ 1.   , 30.015,  0.45 ,  0.   ,  0.   ,  0.   ])], [array([1.]), array([ 1.   , 30.015,  0.45 ,  0.   ,  0.   ,  0.   ]), array([ 1.   , 30.015,  0.45 ,  0.   ,  0.   ,  0.   ]), array([ 1.   , 30.015,  0.45 ,  0.   ,  0.   ,  0.   ])], [array([ 1.   , 30.015,  0.45 , -0.   ,  0.   ,  0.   ]), array([ 1.   , 30.015,  0.45 , -0.   ,  0.   ,  0.   ]), array([ 1.   , 30.015,  0.45 , -0.   ,  0.   ,  0.

<br>

<b> Code fragment for T1.2 stability analysis </b>

In [65]:
# compute eigenvalues
A = np.array(A).astype(float)
eigen_values = np.linalg.eigvals(A)

print('eigenvalues : ', eigen_values)

# compute Jordan form
# Hint: To compute the Jordan form of the matrix M, use the command 'M.jordan_form()', which can only be applied if M is a Sympy matrix.
A_jordan = sp.Matrix(A).jordan_form()
print('\nJordan form: ')
sp.pprint(A_jordan)

 [[  0.       0.       0.       1.       0.       0.       0.       0.       0
  [  0.       0.       0.       0.       1.       0.       0.       0.       0
  [  0.       0.       0.       0.       0.       1.       0.       0.       0
  [  0.       0.       0.      -0.015    0.       0.       6.9367   6.9367   0
  [  0.       0.       0.       0.      -0.015    0.      -6.9367   6.9367   0
  [  0.       0.       0.       0.      -0.015    0.       0.       0.       0
  [  0.       0.       0.       0.       0.       0.       0.       0.       0
  [  0.       0.       0.       0.       0.       0.       0.       0.       0
  [  0.       0.       0.       0.       0.       0.       0.       0.       0
  [  0.       0.       0.       0.       0.       0.       0.       0.       0
  [  0.       0.       0.       0.       0.       0.       0.       0.       0
 [  0.       0.       0.       0.       0.       0.       0.       0.       0.

.       0.       0.       0.    ]
.       0.       

<br>

<b> Code fragment for T1.3 forced responses </b>

In [ ]:
# extracting the rolling dynamics
A_rolling = ...
B_rolling = ...
C_rolling = ...
D_rolling = ...
sys_rolling = ...

# plot impulse response
t_impulse,y_impulse = ...

# plot step response
t_step,y_step = ...

# plot sinusoidal response
t_sin,y_sin = ...

# show the final value of the impulse response
print('The final value of the the rolling steady-state error in the impulsive input case is %.5f' % y_impulse[-1])

<br>

<b> Code fragment for T1.4 proportional control </b>

In [ ]:
def proportional_control(A,B,C,D,k):
    # A,B,C,D: system matrices; k: feedback gain
    # extracting the rolling dynamics
    A_rolling = ...
    B_rolling = ...
    ...
    A_rolling_closed_loop = ...

    print("\n When the gain k is %.3f, the results are as follows." % k)
        
    # Bode plot
    ...
    
    # initial condition response
    t_pc, y_pc = ...

    
# normal case 
proportional_control(A,B,C,D,...)
# choose a stabilizing gain
...
# choose a non-stabilizing gain
...